In [2]:
!pip install boxx > .null

In [4]:
import tensorflow as tf
import os
import numpy as np
import math, numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt
import boxx
import json
from pandas.io.json import json_normalize
import pandas as pd 
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import keras
from tensorflow.keras.optimizers import RMSprop
from pathlib import Path

In [6]:
Path('/content/weights').mkdir(parents=True, exist_ok=True)

# Setting up dataset

In [5]:
!git clone https://github.com/Sern5011/Automatic-item-checkout-system-for-supermarkets-based-on-object-detection-and-recognition.git

Cloning into 'Automatic-item-checkout-system-for-supermarkets-based-on-object-detection-and-recognition'...
remote: Enumerating objects: 83798, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 83798 (delta 4), reused 13 (delta 3), pack-reused 83777
Receiving objects: 100% (83798/83798), 12.63 GiB | 22.60 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Checking out files: 100% (83745/83745), done.


In [6]:
base_dir = './Automatic-item-checkout-system-for-supermarkets-based-on-object-detection-and-recognition/dataset'
train_dir = os.path.join(base_dir, 'train2019')
test_dir = os.path.join(base_dir, 'test/test2019')
val_dir = os.path.join(base_dir, 'val2019')

In [7]:
os.path.exists(test_dir)

True

In [8]:
train_js = boxx.loadjson('./Automatic-item-checkout-system-for-supermarkets-based-on-object-detection-and-recognition/dataset/instances_train2019.json')
val_js = boxx.loadjson('./Automatic-item-checkout-system-for-supermarkets-based-on-object-detection-and-recognition/dataset/instances_val2019.json')
test_js = boxx.loadjson('./Automatic-item-checkout-system-for-supermarkets-based-on-object-detection-and-recognition/dataset/test/instances_test2019.json')
boxx.tree(train_js, deep=1)

└── /: dict  6
    ├── info: dict  7
    ├── licenses: list  1
    ├── categories: list  200
    ├── __raw_Chinese_name_df: list  200
    ├── images: list  53739
    └── annotations: list  53739


In [9]:
boxx.tree(val_js, deep=1)

└── /: dict  6
    ├── info: dict  7
    ├── licenses: list  1
    ├── categories: list  200
    ├── __raw_Chinese_name_df: list  200
    ├── images: list  6000
    └── annotations: list  73602


In [10]:
boxx.tree(test_js, deep=1)

└── /: dict  6
    ├── info: dict  7
    ├── licenses: list  1
    ├── categories: list  200
    ├── __raw_Chinese_name_df: list  200
    ├── images: list  24000
    └── annotations: list  294333


In [12]:
train_df = pd.DataFrame(train_js['images'], dtype=str)

In [13]:
train_df

,file_name,width,height,id
0,038900004095_camera0-13.jpg,2592,1944,0
1,038900004095_camera0-18.jpg,2592,1944,1
2,038900004095_camera0-11.jpg,2592,1944,2
3,038900004095_camera0-10.jpg,2592,1944,3
4,038900004095_camera0-17.jpg,2592,1944,4
...,...,...,...,...
53734,6951384903194~B_camera3-4.jpg,2592,1944,53734
53735,6951384903194~B_camera3-35.jpg,2592,1944,53735
53736,6951384903194~B_camera3-5.jpg,2592,1944,53736
53737,6951384903194~B_camera3-6.jpg,2592,1944,53737


In [14]:
train_df = train_df.assign(code=train_df['file_name'].str[:12])

In [15]:
train_df

,file_name,width,height,id,code
0,038900004095_camera0-13.jpg,2592,1944,0,038900004095
1,038900004095_camera0-18.jpg,2592,1944,1,038900004095
2,038900004095_camera0-11.jpg,2592,1944,2,038900004095
3,038900004095_camera0-10.jpg,2592,1944,3,038900004095
4,038900004095_camera0-17.jpg,2592,1944,4,038900004095
...,...,...,...,...,...
53734,6951384903194~B_camera3-4.jpg,2592,1944,53734,695138490319
53735,6951384903194~B_camera3-35.jpg,2592,1944,53735,695138490319
53736,6951384903194~B_camera3-5.jpg,2592,1944,53736,695138490319
53737,6951384903194~B_camera3-6.jpg,2592,1944,53737,695138490319


In [16]:
test_df = pd.DataFrame(test_js['images'], dtype=str)

In [17]:
test_df

,file_name,width,height,id,level
0,20180824-15-44-39-474.jpg,1831,1831,3,easy
1,20180824-15-44-47-474.jpg,1834,1834,4,easy
2,20180824-15-44-56-474.jpg,1839,1839,5,easy
3,20180824-15-50-57-480.jpg,1827,1827,6,easy
4,20180824-15-51-04-480.jpg,1831,1831,7,easy
...,...,...,...,...,...
23995,20181009-14-48-57-3045.jpg,1825,1825,30527,hard
23996,20181009-14-49-17-3045.jpg,1832,1832,30528,hard
23997,20181010-10-46-43-3093.jpg,1838,1838,30532,hard
23998,20181010-10-47-10-3093.jpg,1809,1809,30533,hard


In [18]:
val_df = pd.DataFrame(val_js['images'], dtype=str)

In [19]:
val_df

,file_name,width,height,id,level
0,20180827-16-07-06-756.jpg,1850,1850,220,easy
1,20180827-16-07-10-756.jpg,1840,1840,221,easy
2,20180827-16-24-37-306.jpg,1826,1826,222,easy
3,20180827-16-24-44-306.jpg,1829,1829,223,easy
4,20180827-16-24-53-306.jpg,1826,1826,224,easy
...,...,...,...,...,...
5995,20181024-15-23-05-131.jpg,1813,1813,30609,hard
5996,20181024-15-37-14-136.jpg,1821,1821,30613,hard
5997,20181024-15-37-31-136.jpg,1832,1832,30614,hard
5998,20181024-15-37-46-136.jpg,1830,1830,30615,hard


In [20]:
# Updated to do image augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='file_name',
        y_col='code',
        directory=train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=100,
        seed=42,
        shuffle=True,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='file_name',
        y_col='level',
        directory=test_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=100,
        seed=42,
        shuffle=True,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col='file_name',
        y_col='level',
        directory=val_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=100,
        seed=42,
        shuffle=True,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

Found 53739 validated image filenames belonging to 201 classes.
Found 24000 validated image filenames belonging to 3 classes.
Found 6000 validated image filenames belonging to 3 classes.


# VGG16

In [21]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(150, 150, 3))



# Load the CONV layers of VGG16 model and add the FC layers

model = models.Sequential()
model.add(conv_base)
#model.pop()
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

58892288/58889256 [==============================] - 1s 0us/step


In [22]:
conv_base.summary()

for layer in conv_base.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in conv_base.layers:
    print(layer, layer.trainable)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


In [25]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
model.evaluate(test_generator)

In [ ]:
model.evaluate(train_generator)

 16/538 [..............................] - ETA: 4:00:23 - loss: 5.3033 - acc: 0.9882

In [26]:
model.evaluate(val_generator)

60/60 [==============================] - 1529s 25s/step - loss: 1.0986 - acc: 0.6413


[1.098612904548645, 0.6404999494552612]